Домашнее задание 2: библиотека Numpy и линейная алгебра

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import ndimage  # спецификатор для работы с изображениями
from scipy import misc
import scipy.stats as sps

from PIL import Image

from random import randint

%matplotlib inline

Что такое NumPy? - это питоновская библиотека для работы с массивами 
Зачем используется и почему работает быстрее? - сам по себе питон не адаптирован к простой работе с массивами, а она адаптирована 
Что такое линейная алгебра? - это раздел математики, в котором оперируют матрицами
Как она связана с машинным обучением и глубоким обучением? - в машинном обучении мы будем оперировать матрицами

Задание 1

В первой задаче вам предлагается перемножить две квадратные матрицы двумя способами -- без использования пакета numpy и с ним.

In [33]:
# Для генерации матриц используем фукнцию random -- она используется для генерации случайных объектов 
# функция sample создает случайную выборку. В качестве аргумента ей передается кортеж (i,j), здесь i -- число строк,
# j -- число столбцов.
a = np.random.sample((10,10))
b = np.random.sample((10,10)) 
# выведите размерность (ранг) каждой матрицы с помощью функции ndim.
print(a.ndim)
print(b.ndim)
# Используйте функцию shape, что она вывела?
print(a.shape) # shape даёт число элементов по каждому из направлений матрицы
print(b.shape)
# ========
# тут можно писать код

def sc_matrix(matrix_first, matrix_second):
    ''' функция скалярного перемножения 2 матриц без np '''
    b = len(matrix_first[0]) != len(matrix_second)
    if b:
        return print('оу оу оу, такие матрицы перемножать скалярно не выйдет, число столбцов первой матрицы не равно числу строк второй')
    else:
        result_matrix = [[0 for el in range(0, len(matrix_second[0]))] for le in range(0, len(matrix_first))]
        for i in range(0, len(matrix_first)):
            for j in range(0, len(matrix_second[0])):
                for r in range(0, len(matrix_first[0])):
                    result_matrix[i][j] += matrix_first[i][r]*matrix_second[r][j]
        return np.array(result_matrix)
    
    pass

def sc_matrix_np(matrix_first, matrix_second):
    try:
        return np.dot(matrix_first, matrix_second)
    except ValueError as ve:
        return print(f'ошибка: {ve}')

# ========
print(f'{a}\n')
print(b)

2
2
(10, 10)
(10, 10)
[[0.84520925 0.40791491 0.32077967 0.41507512 0.75471962 0.30716961
  0.33098834 0.54266157 0.96077901 0.42757194]
 [0.77773526 0.48804343 0.96354581 0.88930252 0.88764659 0.84250388
  0.77120461 0.01979344 0.62295968 0.38562129]
 [0.2769033  0.99152331 0.75514006 0.32234728 0.74807766 0.5129068
  0.72998858 0.46953136 0.48256934 0.6215709 ]
 [0.56922237 0.49905051 0.96575695 0.27757723 0.53389446 0.86224022
  0.52805703 0.36781906 0.89113288 0.93690659]
 [0.02366935 0.86870779 0.40452799 0.00762895 0.99078913 0.90892077
  0.67334653 0.13328948 0.78849645 0.78543573]
 [0.46460068 0.80591729 0.12896041 0.46071474 0.76540201 0.76965662
  0.06506401 0.93983949 0.78770999 0.51785941]
 [0.03882512 0.87922197 0.12161561 0.0471281  0.69672227 0.75582873
  0.805913   0.28575171 0.32927148 0.21037209]
 [0.8428939  0.4961625  0.18914602 0.44802361 0.68037774 0.10507797
  0.86498916 0.49234002 0.54863643 0.8217503 ]
 [0.83965772 0.12741    0.17987985 0.95704739 0.94150047 0.

In [45]:
%%timeit
# засечем время работы функции без NumPy
sc_matrix(a,b)

1.85 ms ± 69.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
%%timeit
# засечем время работы функции с NumPy
sc_matrix_np(a,b)

4.33 µs ± 320 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Разница $10^3$ нумпай впереди


Задание 2

Напишите функцию, которая по данной последовательности $\{A_i\}_{i=1}^n$ строит последовательность $S_n$, где $S_k = \frac{A_1 + ... + A_k}{k}$. Аналогично -- с помощью библиотеки NumPy и без нее. Сравните скорость, объясните результат.

In [ ]:
n = int(input('Enter n : '))
#a_list = np.random.sample((n,1))
#print(a_list)
# без библиотеки нумпай
a_list_sec = sps.uniform.rvs(size=10 ** 7) #[randint(0, 10) for i in range(0, n)] 
print(a_list_sec)
def wo_np_list(a_list):
    a_sum = 0
    s_list = [0 for i in range(0, n)]
    for i, j in enumerate(a_list_sec):
        a_sum += j
        s_list[i] = round(a_sum / (i+1), 2)
    return s_list

# с библиотекой нумпай
def w_np_list(a_list):
    a_list_sec = np.array(a_list_sec)
    a_list_test = np.array([round(a_list_sec[:i].sum()/(i),2) for i in range(1, n+1)])
    return a_list_test
